In [ ]:
import pandas as pd
from backtesting import Strategy
from backtesting.lib import crossover, TrailingStrategy
from backtesting import Backtest
from backtesting.test import SMA

import numpy as np

data = pd.read_csv("../data/BTC-USD.csv", parse_dates=[0], date_parser=pd.to_datetime, index_col = [0], header = 0)

# data = pd.read_csv("BTC-USD-100day.csv", parse_dates=[0], date_parser=pd.to_datetime, index_col = [0], header = 0)


In [ ]:
# def SMA(values, n):
#     """
#     Return simple moving average of `values`, at
#     each step taking into account `n` previous values.
#     """
#     return pd.Series(values).rolling(n).mean()

class BTCStrategy(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 8
    sltr=500e-4

    def init(self):
        # Precompute the two moving averages
        super().init()
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        
    
    def next(self):

        super().next()

        sltr = self.sltr
        if len(self.trades) > 0:
            print("Active Trade: " + str(len(self.trades)))

        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.High[-1] * (1-sltr))
                print("entry_time: " + str(trade.entry_time) + " ,sl: " + str(trade.sl))
            # else:
            #     trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + sltr) 

        if (self.data.Open[-1] > self.sma1[-2] and 
        self.sma1[-2] > self.sma2[-2] and
        self.data.Open[-1] > self.data.Low[-2] and 
        self.data.Low[-2] > self.data.Low[-3] and
        self.data.Low[-3] > self.data.Low[-4]):
            print ("Open[-1]: " + str(self.data.Open[-1]) +
            " Low[-2]: " + str( self.data.Low[-2]) +
            " Low[-3]: " + str( self.data.Low[-3]) +
            " Low[-4]: " + str( self.data.Low[-4]) +
            " sma1: " + str(self.sma1[-2]) +
            " sma2: " + str(self.sma2[-2])
            )
            self.buy(limit=self.data.Open[-1], sl=self.data.Low[-2])


In [ ]:
bt = Backtest(data, BTCStrategy, cash=1000000, commission=.002)
stats = bt.run()
stats
# stats['_equity_curve']


In [ ]:
bt.plot()